# 05 - Model Evaluation

This notebook evaluates model performance and identifies areas for improvement.

## Objectives:
- Calculate performance metrics
- Generate confusion matrix and classification reports
- Visualize predictions vs actual values
- Residual analysis
- Feature importance analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Load the trained model
model = joblib.load('../models/best_model.pkl')
print("Model loaded successfully")

In [ ]:
# Load test data
# X_test = ...
# y_test = ...
# 
# # Make predictions
# y_pred = model.predict(X_test)

In [ ]:
# Calculate performance metrics
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# 
# print(f"Mean Absolute Error: {mae:.4f}")
# print(f"Root Mean Squared Error: {rmse:.4f}")
# print(f"R² Score: {r2:.4f}")

In [ ]:
# Visualize predictions vs actual
# plt.figure(figsize=(10, 6))
# plt.scatter(y_test, y_pred, alpha=0.5)
# plt.xlabel('Actual Values')
# plt.ylabel('Predicted Values')
# plt.title('Predictions vs Actual Values')
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
# plt.show()

In [ ]:
# Feature importance
# if hasattr(model, 'feature_importances_'):
#     feature_importance = pd.DataFrame({
#         'feature': X_test.columns,
#         'importance': model.feature_importances_
#     }).sort_values('importance', ascending=False)
#     
#     plt.figure(figsize=(10, 6))
#     sns.barplot(data=feature_importance.head(20), x='importance', y='feature')
#     plt.title('Top 20 Feature Importance')
#     plt.show()

In [ ]:
# Residual analysis
# residuals = y_test - y_pred
# plt.figure(figsize=(10, 6))
# plt.hist(residuals, bins=30, edgecolor='black')
# plt.xlabel('Residuals')
# plt.ylabel('Frequency')
# plt.title('Distribution of Residuals')
# plt.show()